In [8]:
import cv2



capture = cv2.VideoCapture(0)

while(True):
    ret,frame = capture.read(0)
    cv2.imshow('image',frame)
    
    if cv2.waitKey(1) & 0xFF ==ord('q'):
        break
    
    
    if ret == False:
        break
    cv2.imwrite('kang'+str(i)+'.jpg',frame)
    
        
capture.release()

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    exit(0)


frameFrequency=25


total_frame = 0
id = 0
while True:
    if id<5:
        
        
        
        ret, frame = cap.read(0)
        cv2.imshow('image',frame)
        if ret is False:
            break
        total_frame += 1
        if total_frame%frameFrequency == 0:
            id += 1
            image_name = "image" + str(id) +'.jpg'
            cv2.imwrite(image_name, frame)
            print(image_name)
            
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

image1.jpg
image2.jpg
image3.jpg
image4.jpg
image5.jpg


In [27]:
import cv2
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def draw_rect(img, faces, sf=20):
    for x, y, w, h in faces:
        x = x - sf
        y = y - 2*sf
        cv2.rectangle(img, (x,y), (x+w+2*sf, y+h+3*sf), (255, 0 , 0), 3)

        
        
       
cap = cv2.VideoCapture(0)



frameFrequency=25
total_frame = 0
id = 0

while cap.isOpened():
    ret, frame = cap.read(0)

    if ret is False:
        break

    faces = face_detector.detectMultiScale(frame, 1.1, 4)
    print(faces)

    draw_rect(frame, faces)
    
    total_frame += 1
    if total_frame%frameFrequency == 0:
        id += 1
        image_name = "user" + str(id) +'.jpg'
        cv2.imwrite(image_name, frame)
        print(image_name)
    # Display the output
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if id >=5:        
        break

cap.release()

[[245 168 184 184]]
[[247 171 181 181]]
[[247 170 181 181]]
[[243 169 185 185]]
[[245 171 182 182]]
[[244 169 183 183]]
[[242 169 186 186]]
[[245 171 181 181]]
[[243 169 184 184]]
[[244 170 182 182]]
[[246 171 180 180]]
[[244 171 181 181]]
[[244 171 181 181]]
[[243 170 183 183]]
[[241 169 184 184]]
[[243 171 180 180]]
[[237 168 187 187]]
[[237 168 186 186]]
[[234 167 187 187]]
[[234 168 185 185]]
[[229 165 188 188]]
[[228 164 190 190]]
[[227 163 189 189]]
[[227 165 186 186]]
[[227 165 185 185]]
user1.jpg
[[223 164 189 189]]
[[225 165 185 185]]
[[224 164 187 187]]
[[224 164 187 187]]
[[224 165 186 186]]
[[223 163 188 188]]
[[223 163 188 188]]
[[222 163 188 188]]
[[223 163 188 188]]
[[222 162 189 189]]
[[223 163 187 187]]
[[224 163 186 186]]
[[224 163 187 187]]
[[226 164 185 185]]
[[227 164 184 184]]
[[226 163 187 187]]
[[227 164 185 185]]
[[226 163 185 185]]
[[228 163 185 185]]
[[229 164 185 185]]
[[230 165 184 184]]
[[229 165 184 184]]
[[231 166 182 182]]
[[231 165 182 182]]
[[231 164 

### This jupyter notebook is to match face of a given photo with all the faces from photos folder 

In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

In [2]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('photos') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


### Saving data into data.pt file

In [3]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

### Matching face id of the given photo with available data from data.pt file

In [10]:

def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))





import cv2
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    exit(0)


frameFrequency=25


total_frame = 0
id = 0
while True:

        
        ret, frame = cap.read(0)
        cv2.imshow('image',frame)
        if ret is False:
            break
        total_frame += 1
        if total_frame%frameFrequency == 0:
            id += 1
            image_name = "user" + str(id) +'.jpg'
            cv2.imwrite(image_name, frame)
            print(image_name)
            
        if cv2.waitKey(1) & 0xFF ==ord('q'):
            break
        if id >=5:
            break

    
cap.release()
cv2.destroyAllWindows()

# import cv2

# cam = cv2.VideoCapture(0)

# cv2.namedWindow("test")

# img_counter = 0

# while True:
    
#     ret, frame = cam.read()
#     if not ret:
#         print("failed to grab frame")
#         break
#     cv2.imshow("test", frame)

#     k = cv2.waitKey(1)
#     if k%256 == 27:
#         # ESC pressed
# #         print("Escape hit, closing...")
#         break
#     elif k%256 == 32:
#         # SPACE pressed
#         img_name = "user{}.png".format(img_counter)
#         cv2.imwrite(img_name, frame)
#         print(" image {} clicked ".format(img_name))
#         img_counter += 1
#     if img_counter>5:
#         break
        
    

# cam.release()

# cv2.destroyAllWindows()


































z=0
total_images_to_be_clicked=5
for i in range(1,6):
    name = "user" +str(i) +".jpg"
# name="user3.jpg"
    result = face_match(name, 'data.pt')
    
    z = z+ result[1]
    
z=z/total_images_to_be_clicked
if(z>0.5):
    print("Image didn't match")
elif(z<=0.5):
    print("Image matched with existing user")

print('Face matched with: ',result[0], 'With distance: ',result[1])

user1.jpg
user2.jpg
user3.jpg
user4.jpg
user5.jpg
Image didn't match
Face matched with:  angelina_jolie With distance:  1.2905322313308716


In [12]:
z= 0
for i in range(1,6):
    name = "user" + str(i) +".jpg"
    result = face_match(name,'data.pt')
    print(result)
    z= result[1] +z
    print(z)
    
print(z/5)

('angelina_jolie', 1.3243240118026733)
1.3243240118026733
('angelina_jolie', 1.3289611339569092)
2.6532851457595825
('angelina_jolie', 1.3494564294815063)
4.002741575241089
('angelina_jolie', 1.3276314735412598)
5.330373048782349
('angelina_jolie', 1.3513246774673462)
6.681697726249695
1.336339545249939


In [4]:
import cv2
face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def draw_rect(img, faces, sf=20):
    for x, y, w, h in faces:
        x = x - sf
        y = y - 2*sf
        cv2.rectangle(img, (x,y), (x+w+2*sf, y+h+3*sf), (255, 0 , 0), 3)

        
        
       
cap = cv2.VideoCapture(0)



frameFrequency=25
total_frame = 0
id = 0

while cap.isOpened():
    ret, frame = cap.read(0)

    if ret is False:
        break

    faces = face_detector.detectMultiScale(frame, 1.1, 4)
    print(faces)

    draw_rect(frame, faces)
    
    total_frame += 1
    if total_frame%frameFrequency == 0:
        id += 1
        image_name = "user" + str(id) +'.jpg'
        cv2.imwrite(image_name, frame)
        print(image_name)
    # Display the output
    cv2.imshow('Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    if id >=5:        
        break

cap.release()
cv2.destroyAllWindows()



def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))



z=0
total_images_to_be_clicked=5
for i in range(1,6):
    name = "user" +str(i) +".jpg"
# name="user3.jpg"
    result = face_match(name, 'data.pt')
    
    z = z+ result[1]
    
z=z/total_images_to_be_clicked
if(z>0.8):
    print("Image didn't match")
elif(z<=0.8):
    print("Image matched with existing user")

print('Face matched with: ',result[0], 'With distance: ',z)

[[224 171 244 244]]
[[224 170 246 246]]
[[221 168 252 252]]
[[218 164 261 261]]
[[218 164 261 261]]
[[226 168 250 250]]
[[226 168 250 250]]
[[226 167 252 252]]
[[227 166 254 254]]
[[230 166 249 249]]
[[230 166 249 249]]
[[229 166 250 250]]
[[231 166 246 246]]
[[231 166 246 246]]
[[233 166 244 244]]
[[239 169 236 236]]
[[240 169 235 235]]
[[243 172 230 230]]
[[242 171 230 230]]
[[243 170 231 231]]
[[247 172 224 224]]
[[247 173 224 224]]
[[247 171 226 226]]
[[250 173 219 219]]
[[250 173 220 220]]
user1.jpg
[[250 174 218 218]]
[[252 174 217 217]]
[[251 174 218 218]]
[[251 174 218 218]]
[[251 174 218 218]]
[[251 175 217 217]]
[[251 174 216 216]]
[[253 176 213 213]]
[[252 175 214 214]]
[[255 175 211 211]]
[[254 170 216 216]]
[[255 169 217 217]]
[[255 170 217 217]]
[[255 169 221 221]]
[[257 169 221 221]]
[[261 171 216 216]]
[[263 172 213 213]]
[[261 171 215 215]]
[[258 169 218 218]]
[[257 170 217 217]]
[[248 169 219 219]]
[[248 172 214 214]]
[[243 170 219 219]]
[[244 171 217 217]]
[[243 170 

'kashif'